In [4]:
import pandas as pd
import numpy as np


pd.set_option('display.max_rows', 999)
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

from camelot import utils


### LOA 2021

In [42]:
# test - '480-487'
### full - '339-844'

dd = camelot.read_pdf('../files/loa_2021.pdf', pages='482-483',kind='textedge',flavor='stream')

In [39]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [40]:
df

,0,1,2,3
0,PROGRAMA: 0150 PROCESSO LEGISLATIVO,,,1.276.109.060
1,AÇÃO,GRUPO DESPESA,FONTE DE RECURSO,
2,01.031.0150.1215\nEXPANSÃO E ADEQUAÇÃO DAS INS...,,,10
3,DE JULHO,,,
4,,OUTRAS DESPESAS,,
5,,,,10
6,,CORRENTES,,
7,,,TESOURO DO ESTADO,10
8,"OBRAS, ADAPTAÇÕES E/OU REFORMAS NO PALÁCIO 9 D...",,,
9,INDICADOR DE,,,


In [68]:
mask = df[0].str.contains('ÓRGÃO:').replace(np.nan,False)

df_orgao = df[mask]
df_orgao[2] = "org"

mask = df[0].str.contains('UNIDADE:').replace(np.nan,False)

df_unidade = df[mask]
df_unidade[2]='uni'

mask = df[0].str.contains('PROGRAMAÇÃO DA UNIDADE ORÇAMENTÁRIA').replace(np.nan,False)

df_programacao = df[mask]
df_programacao[2]='prog'

# indexes = list(df_programas.index)

In [70]:
df_orgao

,0,1,2,3,4,5
215,ÓRGÃO: 02000 - TRIBUNAL DE CONTAS DO ESTADO\...,NaN,org,NaN,NaN,NaN
281,ÓRGÃO: 02000 - TRIBUNAL DE CONTAS DO ESTADO,,org,,NaN,NaN


In [46]:
df_include = pd.concat([df_orgao, df_unidade,df_programacao], axis=0).sort_index()

includes = df_include[2].tolist()

l3=[]
for i in range(len(includes)-2):
    if (includes[i]=='org') & (includes[i+1]=='uni') & (includes[i+2]=='prog'):
        l3.append(1)
    else:
        l3.append(0)
l3.append(1)
l3.append(1)

l4=[]        
for i in range(len(includes)-2):       
    if (includes[i-1]=='org') & (includes[i]=='uni') & (includes[i+1]=='prog'):
        l4.append(1)
    else:
        l4.append(0)
l4.append(1)
l4.append(1)

l5=[]
for i in range(len(includes)-2):       
    if (includes[i-2]=='org') & (includes[i-1]=='uni') & (includes[i]=='prog'):
        l5.append(1)
    else:
        l5.append(0)    
l5.append(1)
l5.append(1) 

df_include[3]=l3
df_include[4]=l4
df_include[5]=l5

df_include[6] = df_include[[3,4,5]].any(axis=1)
df_include[1]=df_include[2]
df_include[2]=df_include[6]
df_include = df_include.drop(columns=[3,4,5,6,7,8])

KeyError: '[7 8] not found in axis'

In [93]:
indexes = df_include.index.tolist()
ranges = df_include[2].tolist()

exclude_ranges=[]
for i in range(len(ranges)-1):
    if (ranges[i]==False) & (ranges[i+1]==True):
        exclude_ranges.append([indexes[i],indexes[i+1]])
    elif (ranges[i]==False) & (ranges[i+1]==False):
        exclude_ranges.append([indexes[i],indexes[i+1]])

In [94]:
df2 = df.copy()

for exclude in exclude_ranges:
    print(exclude[0],exclude[1])
    df2.drop(df.index[exclude[0]:exclude[1]], inplace=True)

3 96
197 256
321 410
515 552
599 678
781 857
962 1193
1193 1425
2376 2810
2810 3180
4546 5002
5002 5202
6635 6806
6806 6928
7214 7438
7438 7624
7918 8205
8205 8311
8856 9167
9167 9291
9952 10264
10264 10531
11139 11487
11487 11812
12688 12802
12802 12922
13155 13306
13306 13450
13649 14275
14275 14750
16043 16087
16087 16131
16182 16336
16336 16462
16868 17049
17049 17217
17683 18010
18010 18234
18740 18919
18919 19092
19885 20018
20018 20116
20252 20371
20371 20478
20631 20711
20711 20779
20874 20977
20977 21074
21229 21745
21745 22025
23089 23127
23127 23161
23198 23227
23227 23257


In [95]:
mask = df2[0].str.contains('ÓRGÃO:').replace(np.nan,False)
df_orgao = df2[mask]
df_orgao=df_orgao[[0]]
# df_orgao[1] = "org"

mask = df2[0].str.contains('UNIDADE:').replace(np.nan,False)
df_unidade = df2[mask]
df_unidade=df_unidade[[0]]
df_unidade[1]='uni'

mask = df2[0].str.contains('PROGRAMAÇÃO DA UNIDADE ORÇAMENTÁRIA').replace(np.nan,False)
df_programacao = df2[mask]
df_programacao=df_programacao[[0]]
df_programacao[1]='prog'

# indexes = list(df_programas.index)

122

In [108]:
indexes = df_orgao.index.tolist()
orgs = df_orgao[0].tolist()
unidades = df_unidade[0].tolist()

df_aux = pd.DataFrame([1])
df_final = pd.DataFrame()

for i in range(len(unidades)):   
    
#     print(unidades[i])

    if i == len(indexes)-1:
        df_f = df2.loc[indexes[i]:]
        mask = df_f[0].str.contains('PROGRAMA:').replace(np.nan,False)
        programas = df_f[mask][0].tolist()
        prog_index = df_f[mask][0].index.tolist()
        
        
        for j in range(len(programas)):
#             print(programas[j])
            
            if j == len(programas)-1:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:]    
                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))
                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                
            else:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:prog_index[j+1]-1]
                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                mask = df_ff[0]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))
                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)
                        
                        
                        
                        
                        
    else:
        
        df_f = df2.loc[indexes[i]:indexes[i+1]]
        mask = df_f[0].str.contains('PROGRAMA:').replace(np.nan,False)
        programas = df_f[mask][0].tolist()
        prog_index = df_f[mask][0].index.tolist()

        for j in range(len(programas)):
#             print(programas[j])
            if j == len(programas)-1:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:]   
#                 display(df_ff)
#                 display(df_ff)

                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))
                
                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                
            else:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:prog_index[j+1]-1]
                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                mask = df_ff[0]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))

                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j] 
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)
            
#             df_final = pd.concat([df_final,df_aux],axis=0)
            
        
#         print(indexes[i], orgs[i], unidades[i])

In [116]:
df_final.to_excel('pre_ldo_4.xlsx', index=False, encoding='utf-8')

### LDO_4 Revival

In [117]:
df = pd.read_excel('pre_ldo_4.xlsx', encoding='utf-8')

In [118]:
df[3] = np.where(df[2]=='FONTE DE RECURSO', 'FONTE DE RECURSO',df[3])
df[2] = np.where(df[1]=='GRUPO DESPESA','GRUPO DESPESA',df[2])
df[1] = np.where(df[0]=='AÇÃO', 'AÇÃO',df[1])
df[0] = np.where(df[0]=='AÇÃO', 'NUMERO DA AÇÃO',df[0])

df[5] = np.where(df[0]=='PRODUTO:', df[1],np.nan)
df[6] = np.where(df[0]=='INDICADOR DE\rPRODUTO:', df[1], np.nan)
df[7] = np.where(df[0]=='META:', df[1], np.nan)


df[1] = np.where((df[0]=='PRODUTO:') | (df[0]=='INDICADOR DE\rPRODUTO:') | (df[0]=='META:'), np.nan,df[1])
df[0] = np.where((df[0]=='PRODUTO:') | (df[0]=='INDICADOR DE\rPRODUTO:') | (df[0]=='META:'), np.nan,df[0])


df[5] = np.where(df[0]=='NUMERO DA AÇÃO', 'PRODUTO',df[5])
df[6] = np.where(df[0]=='NUMERO DA AÇÃO', 'INDICADOR DE PRODUTO',df[6])
df[7] = np.where(df[0]=='NUMERO DA AÇÃO', 'META',df[7])

In [119]:
df[0].fillna(method='ffill', inplace=True)
df[1].fillna(method='ffill', inplace=True)
df[2].fillna(method='ffill', inplace=True)
df[5].fillna(method='bfill', inplace=True)
df[6].fillna(method='bfill', inplace=True)
df[7].fillna(method='bfill', inplace=True)

#remove os nulls das fontes de recurso
mask = df[3].notnull()
df = df[mask]

#remove os nulls dos valores
mask = df[4].notnull()
df = df[mask]

# coloca nan nas linhas que nao possuem meta
df[7] = np.where(df[7]=='META', np.nan, df[7])

df = df.drop(columns=[8])

In [120]:
rename = {
    0:'codigo da acao',
    1:'acao',
    2:'grupo despesa',
    3:'fonte de recurso',
    4:'valor',
    5:'produto',
    6:'indicador de produto',
    7:'meta'
}

df = df.rename(columns=rename)

In [121]:
df.columns

Index(['codigo da acao', 'acao', 'grupo despesa', 'fonte de recurso', 'valor',
       'produto', 'indicador de produto', 'meta', 'orgaos', 'unidade',
       'programas', 'numero de acoes'],
      dtype='object')

In [122]:
cols = [
    'orgaos', 'unidade',
    'programas', 
    'codigo da acao', 'acao','numero de acoes',
    'produto','indicador de produto', 
    'grupo despesa', 'fonte de recurso', 
    'meta','valor'
]

df = df[cols]

In [123]:
for col in df.columns:
    if (col!='numero de acoes'):
        df[col] = df[col].str.replace('\r',' ')
    
    
df['valor'] =  df['valor'].str.replace('.','').str.replace(',','.').astype(float)
df['meta']  =  df['meta'].str.replace('.','').str.replace(',','.').astype(float)

In [124]:
df['contador'] = 1

In [125]:
replace_values = {
    'UTRAS DESPESAS ORRENTES':'OUTRAS DESPESAS CORRENTES',
    'NVESTIMENTOS':'INVESTIMENTOS ',
    'ESSOAL E ENCARGOS OCIAI':'PESSOAL E ENCARGOS SOCIAIS',
    'NVERSÕES FINANCEIRAS':'INVERSÕES FINANCEIRAS',
    'ESERVA DE ONTINGÊNCIA':'RESERVA DE CONTINGÊNCIA',
    'UROS E ENCARGOS DA ÍVIDA':'JUROS E ENCARGOS DA DÍVIDA',
    'MORTIZAÇÃO DA DÍVIDA':'AMORTIZAÇÃO DA DÍVIDA'
}


df = df.replace({"grupo despesa": replace_values})

In [126]:
replace_values = {
    'ESOURO DO ESTADO':'TESOURO DO ESTADO',
    'INCULADOS FEDERAIS':'VINCULADOS FEDERAIS',
    'UNDO ESPECIAL DE ESPESA':'FUNDO ESPECIAL DE DESPESA',
    'RÓPRIOS':'PRÓPRIOS',
    'INCULADOS ESTADUAIS':'VINCULADOS ESTADUAIS',
    'UTRAS FONTES - UNDOS':'OUTRAS FONTES - FUNDOS',
    'ESOURO-DOT.INICIAL E RED.SUPLEMENTAR- NTRA':'TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR- INTRA',
    'PERAÇÕES DE RÉDITO':'OPERAÇÕES DE CRÉDITO',
    'UTRAS FONTES - DREM':'OUTRAS FONTES - DREM',
    'ECURSOS INCULADOS-FUNDO SP. DESPESA-INTRA':'RECURSOS VINCULADOS-FUNDO ESP. DESPESA-INTRA',
    'ECURSOS VINCULADOS EDERAIS-INTRA':'RECURSOS VINCULADOS FEDERAIS-INTRA',
    'EC.PROPRIO-ADM.IND.- OT.INIC.CR.SUPL.- NTRA':'REC.PROPRIO-ADM.IND.- DOT.INIC.CR.SUPL.- INTRA',
    'P.CRED.CONTR.EXTERI R-DOT.INIC.CR.SUP- NTRA':'OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA',
    'UTRAS FONTES DE ECURSOS-INTRA':'OUTRAS FONTES DE RECURSOS-INTRA '
    
}


df = df.replace({"fonte de recurso": replace_values})

In [127]:
# df['indicador de produto'] = df['indicador de produto'].str.replace('ERCENTUAL','PERCENTUAL').str.replace('ÚMERO','NÚMERO').str.replace('XTENSÃO','EXTENSÃO').str.replace('ENEFICIÁRIOS','BENEFICIÁRIOS')

# df['indicador de produto'] = df['indicador de produto'].str.replace('ERCENTUAL','PERCENTUAL').str.replace('ÚMERO','NÚMERO')

In [128]:
mask = (df['valor']==32265100)&(df['codigo da acao']=='roposta Orçamentária 2020')

df = df.set_value(df[mask].index,'codigo da acao' ,'10.844.0000.9019')
df = df.set_value(df[mask].index,'acao' ,'PAGAMENTO DA DÍVIDA PÚBLICA EXTERNA')

mask = (df['valor']==51720442)&(df['codigo da acao']=='roposta Orçamentária 2020')

df = df.set_value(df[mask].index,'codigo da acao' ,'10.844.0000.9019')
df = df.set_value(df[mask].index,'acao' ,'PAGAMENTO DA DÍVIDA PÚBLICA EXTERNA')



mask = (df['valor']==28623000)&(df['codigo da acao']=='roposta Orçamentária 2020')

df = df.set_value(df[mask].index,'codigo da acao' ,'10.303.0935.4192')
df = df.set_value(df[mask].index,'acao' ,'ATENDIMENTO HEMOTERÁPICO')




mask = (df['programas']=='PROGRAMA:     0930    ATENDIMENTO INTEGRAL E DESCENTRALIZADO NO SUS NO ESTADO DE SÃO PAULO') & (df['codigo da acao']=='roposta Orçamentária 2020')

df = df.set_value(df[mask].index,'codigo da acao' ,'10.302.0930.5274')
df = df.set_value(df[mask].index,'acao' ,'ASSISTÊNCIA MÉDICA, HOSPITALAR E AMBULATORIAL')



mask = (df['valor']==2202907)&(df['codigo da acao']=='roposta Orçamentária 2020')

df = df.set_value(df[mask].index,'codigo da acao' ,'18.542.2604.2622')
df = df.set_value(df[mask].index,'acao' ,'CONTRAPARTIDAS AMBIENTAIS E CONVÊNIOS')



In [1]:
# df.head(999)

In [2]:
# df = pd.read_excel('ldo_4.xlsx')

In [9]:
# df.groupby(by='acao').sum().sort_values(by='valor',ascending=False)

In [10]:
# df.groupby(by='unidade').sum().sort_values(by='valor',ascending=False)/df.groupby(by='unidade').sum()['valor'].sum()*100


In [322]:
# df.to_excel('ldo_4.xlsx', index=False, encoding='utf-8')

In [324]:
# df.head()